## Imports

In [1]:
import os
import cv2
import pathlib
import itertools
import numpy as np
from matplotlib import pyplot as plt

import torch
import torchvision
import ultralytics

import preprocessing

In [ ]:
# Clone the tensorflow models repository if it doesn't already exist
if "yolov5" in pathlib.Path.cwd().parts:
  while "yolov5" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('yolov5').exists():
  !git clone --depth 1 https://github.com/ultralytics/yolov5

In [ ]:
!cp yolov5/data/VOC.yaml yolov5/VOC.yaml 

In [ ]:
device = 'cuda'
boxes = torch.tensor([[0., 1., 2., 3.]]).to(device)
scores = torch.randn(1).to(device)
iou_thresholds = 0.5

print(torchvision.ops.nms(boxes, scores, iou_thresholds))

In [2]:
ultralytics.checks()

Ultralytics YOLOv8.2.59  Python-3.9.19 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1080, 8192MiB)
Setup complete  (12 CPUs, 15.9 GB RAM, 214.1/222.3 GB disk)


## 1. Create Dataset

In [6]:
# Klassenamen
classes = ["1", "2", "3", "4", "5"]

In [8]:
os.chdir('../')

In [12]:
annotation_dir_test = "./yoloNoUnkData/Test/Annotations/"
annotation_dir_train = "./yoloNoUnkData/Train/Annotations/"

In [11]:
preprocessing.convert_voc_to_yolo(annotation_dir=annotation_dir_test, output_dir=annotation_dir_test, class_names=classes)

In [13]:
preprocessing.convert_voc_to_yolo(annotation_dir=annotation_dir_train, output_dir=annotation_dir_train, class_names=classes)

## 2. Training  

In [ ]:
ls

In [3]:
os.chdir('./yolov5')

In [19]:
cd ./yolov5 # Go into yolov5 directory

Das System kann den angegebenen Pfad nicht finden.


646m 52.5s !python train.py --img 640 --batch 8 --epochs 50 --data noUnkVOC.yaml --weights yolov5l.pt --workers 2 --project ../yoloTrainNoUnk/ --name yolo5large50E640 --patience 10

In [4]:
!python train.py --img 640 --batch 8 --epochs 50 --data noUnkVOC.yaml --weights yolov5l.pt --workers 2 --project ../yoloTrainNoUnk/ --name yolo5large50E640 --patience 10

^C


In [32]:
!python train.py --img 320 --batch 32 --epochs 50 --data myVOC.yaml --weights yolov5l.pt --workers 2 --project ../yoloTrain/ --name yolo5large50E320I --patience 5

^C


In [6]:
!python train.py --img 320 --batch 32 --epochs 50 --data myVOC.yaml --weights ../yoloTrain/yolo5large50E320I/weights/last.pt --workers 2 --project ../yoloTrain/ --name yolo5large50E320I --patience 5

train: weights=../yoloTrain/yolo5large50E320I/weights/last.pt, cfg=, data=myVOC.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=50, batch_size=32, imgsz=320, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data\hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=2, project=../yoloTrain/, name=yolo5large50E320I, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=5, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github:  YOLOv5 is out of date by 3 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5  12be499 Python-3.9.19 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1080, 8192MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weigh

In [7]:
!python val.py --img 1280 --data noUnkVOC.yaml --weights ../yoloTrainNoUnk/yolo5large50E640/weights/best.pt --workers 1 --project ../yoloTrainNoUnk/ --name yolo5large50E640_1280

^C


### Video

Best: yolo5medium50E320_1280 fuer SingleRow

Notes: img = 1280, bei yolo5small50E640I2 ist bisher am besten fuer shortTest.
img = 640, bei yolo5small50E640I2 ist schlecht fuer singleRow.

In [38]:
!python detect.py --img 1280 --device 0 --source ../videoInferenceData/15FPS_singleRow/15FPS_singleRow_test1.mp4  --weights ../yoloTrainNoUnk/yolo5large50E640//weights/best.pt --project ../videoInferenceData/ --name yoloTrainNoUnk/15FPS_singleRow/15FPS_singleRow_test1/yolo5large50E640_1280 --conf-thres 0.35

detect: weights=['../yoloTrainNoUnk/yolo5large50E640//weights/best.pt'], source=../videoInferenceData/15FPS_singleRow/15FPS_singleRow_test1.mp4, data=data\coco128.yaml, imgsz=[1280, 1280], conf_thres=0.35, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=../videoInferenceData/, name=yoloTrainNoUnk/15FPS_singleRow/15FPS_singleRow_test1/yolo5large50E640_1280, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  12be499 Python-3.9.19 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1080, 8192MiB)

Fusing layers... 
Model summary: 267 layers, 46129818 parameters, 0 gradients, 107.7 GFLOPs
video 1/1 (1/1181) C:\Users\Alexej\Desktop\GTSRB\videoInferenceData\15FPS_singleRow\15FPS_singleRow_test1.mp4: 736x1280 1 5, 68.1ms
video 1/1 (2/1181) C:\Users\Alexej\Desktop\G

yoloTrainNoUnk/multiRow/mixClassNoBackroundSlowToFast/yolo5large50E640_1920: Bisher das beste Ergebnis mit 1280 und 800
<br> yolo5large50E640: Speed: 3.0ms pre-process, 222.9ms inference, 1.2ms NMS per image at shape (1, 3, 2560, 2560) FPS: 4.40
<br> yolo5large50E640: Speed: 0.9ms pre-process, 67.5ms inference, 1.0ms NMS per image at shape (1, 3, 1280, 1280) FPS: 14.39
<br> yolo5small50E320: Speed: 1.0ms pre-process, 20.2ms inference, 1.3ms NMS per image at shape (1, 3, 1280, 1280) FPS: 44.48
<br> yolo5medium50E640: Speed: 2.0ms pre-process, 72.0ms inference, 1.7ms NMS per image at shape (1, 3, 1920, 1920) FPS: 13.22
<br> yolo5medium50E640: 
allClassBrightToDark:
Speed: 3.0ms pre-process, 224.6ms inference, 1.3ms NMS per image at shape (1, 3, 2560, 2560)
FPS: 4.37
Speed: 2.0ms pre-process, 134.9ms inference, 1.2ms NMS per image at shape (1, 3, 1920, 1920)
FPS: 7.24
Speed: 0.9ms pre-process, 67.3ms inference, 1.1ms NMS per image at shape (1, 3, 1280, 1280)
FPS: 14.43
<br> 
15FPS_singleRow_test1:
Speed: 3.0ms pre-process, 223.9ms inference, 1.5ms NMS per image at shape (1, 3, 2560, 2560)
FPS: 4.38
Speed: 2.1ms pre-process, 135.2ms inference, 1.5ms NMS per image at shape (1, 3, 1920, 1920)
FPS: 7.20


## 3. Evaluation

In [ ]:
cd .. # Go in root directory

### 3.1 Video evaluation

In [5]:
!python detect.py --source 0  --weights ../yoloTrainNoUnk/yolo5small50E1280I2/weights/best.pt --img 1280 --conf-thres 0.5

detect: weights=['../yoloTrainNoUnk/yolo5small50E1280I2/weights/best.pt'], source=0, data=data\coco128.yaml, imgsz=[1280, 1280], conf_thres=0.6, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  12be499 Python-3.9.19 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1080, 8192MiB)

Fusing layers... 
Model summary: 157 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs
1/1: 0...  Success (inf frames 640x480 at 30.00 FPS)

0: 960x1280 (no detections), 1326.6ms
0: 960x1280 (no detections), 19.0ms
0: 960x1280 (no detections), 17.0ms
0: 960x1280 (no detections), 17.0ms
0: 960x1280 (no detections), 18.5ms
0: 960x1280 (no detections), 16.0ms
0: 960x1280 (no detections), 15.5ms
0: 960x1280 (n

FPS = 1000ms / (11.3ms ) = 88.5 FPS
Speed: 0.4ms pre-process, 9.0ms inference, 1.9ms NMS per image at shape (1, 3, 320, 320)

## 3.2 Simple detection 

In [ ]:
!python detect.py --source ../yoloData/Test/images --weights ./runs/train/exp8/weights/last.pt --img 320 --save-txt --save-conf --line-thickness 1

## 4. Inference

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='./runs/train/exp8/weights/best.pt', force_reload=True)
model.eval()

In [ ]:
fig, ax = plt.subplots(2,4, figsize=(20,10))
imgs = os.listdir('../yoloData/Test/images')

for idx in itertools.product(range(2),range(4)): 
    imgname = np.random.choice(imgs)
    img = cv2.imread(f'../yoloData/Test/images/{imgname}')
    results = model(img)
    ax[idx[0],idx[1]].imshow(cv2.cvtColor(np.squeeze(results.render()), cv2.COLOR_BGR2RGB))

In [ ]:
import glob
from IPython.display import Image, display

for imageName in glob.glob('./runs/detect/exp12/*.jpg'): #assuming JPG
    display(Image(filename=imageName, width=320, height=320))
    print("\n")

In [ ]:
fig, ax = plt.subplots(2,4, figsize=(20,10))
imgs = os.listdir('../yoloData/Test/images')

for idx in itertools.product(range(2),range(4)): 
    imgname = np.random.choice(imgs)
    img = cv2.imread(f'../yoloData/Test/images/{imgname}')
    results = model(img)
    ax[idx[0],idx[1]].imshow(cv2.cvtColor(np.squeeze(results.render()), cv2.COLOR_BGR2RGB))